In [1]:
from pykrx import stock
from typing import Dict, Optional, List, Tuple
import pandas as pd
import numpy as np
from pypfopt import EfficientFrontier
from enum import Enum
import datetime
import uuid

import simulation.config as config
from simulation import asset_position

/opt/anaconda3/envs/pyra/lib/python3.9/site-packages/pykrx/__init__.py:4: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


In [2]:
import time
ticker_list = ['005930', '000020', '035720']
for ticker in ticker_list:
    df = stock.get_market_ohlcv('20181210', '20181212', ticker)
    print(df)
    time.sleep(1)

               시가     고가     저가     종가       거래량       등락률
날짜                                                        
2018-12-10  40450  40650  40000  40200  14892263 -1.831502
2018-12-11  40600  40700  40200  40250  10638766  0.124378
2018-12-12  40250  40700  40150  40450  12024279  0.496894
              시가    고가    저가    종가    거래량       등락률
날짜                                                 
2018-12-10  9590  9710  9520  9660  58399 -0.309598
2018-12-11  9660  9760  9320  9320  82378 -3.519669
2018-12-12  9330  9580  9330  9570  23962  2.682403
               시가     고가     저가     종가     거래량       등락률
날짜                                                      
2018-12-10  22679  22779  22077  22279  331087 -3.058916
2018-12-11  22076  22177  21375  21476  461496 -3.604291
2018-12-12  21475  22277  21174  22078  393740  2.803129


In [3]:
class PykrxDataLoader:
    def __init__(self, fromdate: str, todate: str, market: str = "KOSPI"):
        self.fromdate = fromdate
        self.todate = todate
        self.market = market
    # 주가 데이터 불러오기
    def load_stock_data(self, ticker_list: List, freq: str, delay: float = 1):
        ticker_data_list = []
        for ticker in ticker_list:
            ticker_data = stock.get_market_ohlcv(fromdate=self.fromdate,
                                                 todate=self.todate,
                                                 ticker=ticker,
                                                 freq='d',
                                                 adjusted=True)
            ticker_data = ticker_data.rename(
                columns = {'시가': 'open', '고가': 'high', '저가': 'low',
                             '종가': 'close', '거래량': 'volume',
                             '거래 대금': 'trading_value', '등락률': 'change_pct'}
            )
            ticker_data = ticker_data.assign(ticker=ticker)
            ticker_data.index.name = 'date'
            ticker_data_list.append(ticker_data)
            time.sleep(delay)
        data = pd.concat(ticker_data_list)
        # 잠시 거래를 중단한 주식의 시가, 고가, 저가 보충
        data.loc[data.open ==0,
                    ['open', 'high', 'low']] = data.loc[data.open == 0, 'close']
        # 샘플링을 통해 일 데이터를 다른 주기 데어터로 변환
        if freq != 'd':
            rule = {
                'open': 'first',
                'high': 'max',
                'low': 'min',
                'close': 'last',
                'volume': 'sum',
                # 'trading_value': 'sum'
            }
            data = data.groupby('ticker').resample(freq).apply(
                rule).reset_index(level=0)
        data.__setattr__('frequence', freq)
        return data

In [4]:
fromdate = '2020-01-01'
todate = '2020-12-31'
ticker_list = ['005930', '000020', '035720']

data_loader = PykrxDataLoader(fromdate=fromdate, todate=todate, market='KOSPI')
ohlcv_data = data_loader.load_stock_data(ticker_list=ticker_list, freq='m', delay=1)
ohlcv_data.head(15)


/var/folders/rk/v4lbk0_j1l5ddg0wmz4352880000gn/T/ipykernel_27365/1591028475.py:38: FutureWarning: 'm' is deprecated and will be removed in a future version, please use 'ME' instead.
  data = data.groupby('ticker').resample(freq).apply(


,ticker,open,high,low,close,volume
date,,,,,,
2020-01-31,000020,8340,8960,7830,7910,3671841
2020-02-29,000020,7790,7890,6590,6830,2983877
2020-03-31,000020,6740,7330,4800,6550,4388718
2020-04-30,000020,6610,12450,6420,11250,32556497
2020-05-31,000020,10650,12450,9710,11550,22757189
2020-06-30,000020,11750,18100,10950,16250,80711715
2020-07-31,000020,16100,27000,14400,24400,55867988
2020-08-31,000020,24700,34450,20700,23450,59577001
2020-09-30,000020,23500,29600,20600,21850,28309516


In [5]:
def calculate_return(ohlcv_data: pd.DataFrame):
    close_data = ohlcv_data[['close', 'ticker']].reset_index().set_index(
        ['ticker', 'date'])
    close_data = close_data.unstack(level=0)
    close_data = close_data['close']
    return_data = close_data.pct_change(1) * 100
    return return_data

In [6]:
def get_mean_variance_weights(return_data: pd.DataFrame,
                              risk_aversion: float) -> Optional[Dict]:
    # 수익률 계산
    expected_return = return_data.mean(skipna=False).to_list()
    # 공분산 행렬 계산
    cov = return_data.cov(min_periods=len(return_data))

    if cov.isnull().values.any() or cov.empty:
        return None

    # 평균-분산 최적화
    ef = EfficientFrontier(
        expected_returns=expected_return,
        cov_matrix=cov,
        solver='OSQP'
    )
    ef.max_quadratic_utility(risk_aversion=risk_aversion)
    # 0에 가까운 편입 비중 처리
    weights = dict(ef.clean_weights(rounding=None))
    return weights
    

In [7]:
return_data = calculate_return(ohlcv_data=ohlcv_data)
clean_rd = return_data.dropna()
print(clean_rd)
weights = get_mean_variance_weights(return_data=clean_rd, risk_aversion=3.07)
print(weights)

ticker         000020     005930     035720
date                                       
2020-02-29 -13.653603  -3.900709   8.178485
2020-03-31  -4.099561 -11.900369  -9.593604
2020-04-30  71.755725   4.712042  18.326872
2020-05-31   2.666667   1.400000  43.207603
2020-06-30  40.692641   4.142012   1.518303
2020-07-31  50.153846   9.659091  28.412583
2020-08-31  -3.893443  -6.735751  18.485481
2020-09-30  -6.823028   7.777778 -10.441787
2020-10-31 -18.077803  -2.749141  -9.465426
2020-11-30   6.983240  17.844523  11.514886
2020-12-31   2.610966  21.439280   5.843250
{'000020': 0.0, '005930': 0.766690977448803, '035720': 0.233309022551197}


In [8]:
class OrderType(Enum):
    # 시장가 주문
    MARKET = 1
    # 지정가 주문
    LIMIT = 2
    # 정지 시장가 주문
    STOPMARKET = 3
    # 정지 지정가 주문
    STOPLIMIT = 4
    

In [9]:
class OrderStatus(Enum):
    # 미체결(혹은 부분 체결)
    OPEN = 1
    # 완료
    FILLED = 2
    # 취소
    CANCELLED = 3

In [10]:
class OrderDirection(Enum):
    # 매수
    BUY = 1
    # 매도
    SELL = -1

In [11]:
class Order(object):
    def __init__(self, dt: datetime.date, ticker: str, amount: int,
                 type: Optional[OrderType] = OrderType.MARKET,
                 limit: Optional[float] = None, stop: Optional[float] = None,
                 id: Optional[str] = None) -> None:
        self.id = id if id is not None else uuid.uuid4().hex
        self.dt = dt
        self.ticker = ticker
        self.amount = abs(amount)
        self.direction = OrderDirection.BUY if amount > 0 else OrderDirection.SELL
        self.type = type
        self.limit = limit
        self.stop = stop

        self.status: OrderStatus = OrderStatus.OPEN
        self.open_amount: int = self.amount

In [12]:
class Transaction(object):
    def __init__(self, id: str, dt:datetime.date, ticker: str, amount: int,
                 price: float, direction: OrderDirection,
                 commission_rate: float = config.commission_rate)->None:
        self.id = id
        self.dt = dt
        self.ticker = ticker
        self.amount = amount
        self.price = price
        self.direction = direction
        self.commission_rate = commission_rate

        self.commission = (self.amount * self.price) * self.commission_rate
        self.settlement_value = -self.direction.value * (self.amount * self.price
                                                        ) - self.commission

In [13]:
class Broker(object):
    def __init__(self, slippage_rate: float = config.slippage_rate,
                 volume_limit_rate: float = config.volume_limit_rate):
        self.slippage_rate = slippage_rate
        self.volume_limit_rate = volume_limit_rate

    def calculate_slippage(self, data: Dict, order: Order) -> Tuple[float, int]:
        # 슬리피지를 포함한 거래 가격 계산
        price = data['open']
        simulated_impact = price * self.slippage_rate

        if order.direction == OrderDirection.BUY:
            impacted_price = price + simulated_impact
        else:
            impacted_price = price - simulated_impact

        # 거래가 가능한 수량 계산
        volume = data['volume']
        max_volume = volume * self.volume_limit_rate
        shares_to_fill = min(order.open_amount, max_volume)

        return impacted_price, shares_to_fill
    
    def process_order(self, dt: datetime.date, data: pd.DataFrame,
                      orders: Optional[List[Order]]) -> List[Transaction]:
        if orders is None:
            return []

        # 가격 데이터를 딕셔너리로 변환
        data = data.set_index('ticker').to_dict(orient='index')

        transactions = []
        for order in orders:
            if order.status == OrderStatus.OPEN:
                assert order.ticker in data.keys()
                # 슬리피지 계산
                price, amount = self.calculate_slippage(
                    data=data[order.ticker],
                    order=order
                )
                if amount != 0:
                    # 거래 객체 생성
                    transaction = Transaction(
                        id=order.id,
                        dt=dt,
                        ticker=order.ticker,
                        amount=amount,
                        price=price,
                        direction=order.direction,
                    )
                    transactions.append(transaction)
                    # 거래 객체의 상태와 미체결 수량 업데이트
                    if order.open_amount == transaction.amount:
                        order.status = OrderStatus.FILLED
                    order.open_amount -= transaction.amount

        return transactions

In [14]:
class AssetPosition(object):
    def __init__(self, ticker: str, position: int, latest_price: float, cost: float):
        self.ticker = ticker
        self.position = position
        self.latest_price = latest_price
        self.cost = cost

        self.total_settlement_value = (-1.0) * self.position * self.cost

    def update(self, transaction: Transaction):
        self.total_settlement_value += transaction.settlement_value
        self.position += transaction.direction.value * transaction.amount
        self.cost = (-1.0) * self.total_settlement_value / self.position \
            if self.position != 0 else 0.0

In [15]:
class Account(object):
    def __init__(self, initial_cash: float) -> None:
        self.initial_cash = initial_cash
        self.current_cash = initial_cash

        self.dt = None

        self.portfolio: Dict[str, AssetPosition] = {}
        self.orders: List[Order] = []

        self.transaction_history: List[Dict] = []
        self.portfolio_history: List[Dict] = []
        self.account_history: List[Dict] = []
        self.order_history: List[Dict] = []
        self.weight_history: List[Dict] = []

    @property
    def total_asset(self) -> float:
        # 현재 총 자산 계산
        market_value = 0
        for asset_position in self.portfolio.values():
            market_value += asset_position.latest_price * asset_position.position
        return market_value + self.current_cash

    def update_position(self, transactions: List[Transaction]):
        for tran in transactions:
            asset_exists = tran.ticker in self.portfolio.keys()
            if asset_exists:
                # 기존에 보유 중인 자산 포지션 업데이트
                self.portfolio[tran.ticker].update(transaction=tran)
            else:
                # 처음 보유하는 자산 추가
                new_position = AssetPosition(
                    ticker=tran.ticker, position=tran.direction.value*tran.amount,
                    latest_price=tran.price,
                    cost=abs(tran.settlement_value)/tran.amount
                )
                self.portfolio[tran.ticker] = new_position
            # 현재 현금 업데이트
            self.current_cash += tran.settlement_value
            # 거래 히스토리 업데이트
            self.transaction_history.append(vars(tran))

    def update_portfolio(self, dt: datetime.date, data: pd.DataFrame):
        # 가격 데이터르르 딕셔너리로 변환
        data = data.set_index('ticker').to_dict(orient='index')

        # 자산의 최신 가격 업데이트
        for asset_position in self.portfolio.values():
            assert asset_position.ticker in data.keys()
            asset_position.latest_price = data[asset_position.ticker]['close']

        # 투자 포트폴리오 히스토리 업데이트 (현금과 자산)
        self.portfolio_history.append(
            {'date': dt, 'ticket': 'cash', 'latest_price': self.current_cash}
        )
        self.portfolio_history.extend(
            [{'date': dt} | vars(asset_position)
              for asset_position in self.portfolio.values()]
        )
        # 장부 ㅁ액 히스토리 업데이트
        self.account_history.append(
            {'date': dt, 'current_cash': self.current_cash, 'total_asset': self.total_asset}
        )

    def update_order(self):
        # 완료 상태의 주문
        filled_orders = [order for order in self.orders
                         if order.status == OrderStatus.FILLED]
        # 주문 히스토리 업데이트
        self.order_history.extend([vars(order) for order in filled_orders])

        # 미완료 상태의 주문은 현재 주문으로 유지
        open_orders = [order for order in self.orders
                       if order.status == OrderStatus.OPEN]
        self.orders[:] = open_orders


In [16]:
def order_target_amount(account: Account, dt: datetime.date,
                        ticker: str, target_amount: int) -> Optional[Order]:
    # 투자 포트폴리오의 각 자산 및 보유 수량
    positions = {asset_position.ticker: asset_position.position
                for asset_position in account.portfolio.values()}
    # 사잔의 보유 수량
    position = positions.get(ticker, 0)
    # 거래 수량 계산
    amount = target_amount - position
    if amount != 0:
        # 주문 객체 생성
        return Order(dt=dt, ticker=ticker, amount=amount)
    else:
        return None

def calculate_target_amount(account: Account, ticker: str,
                            target_percent: float, data: pd.DataFrame) -> int:
    assert ticker in data['ticker'].tolist()
    # 총 자산
    total_asset = account.total_asset
    # 자산의 현재 가격
    price = data.loc[data['ticker'] == ticker, 'close'].squeeze()
    # 목표 보유 수량 계산
    target_amount = int(np.fix(total_asset * target_percent / price))
    return target_amount

def order_target_percent(account: Account, dt: datetime.date, ticker: str,
                         target_percent: float, data: pd.DataFrame) -> Optional[Order]:
    # 목표 보유 수량 계산
    target_amount = calculate_target_amount(account=account, ticker=ticker,
                                            target_percent=target_percent, data=data)
    # 목표 수량에 따라 주문
    return order_target_amount(account=account, dt=dt, ticker=ticker, target_amount=target_amount)


